In [1]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
import chainlit as cl

In [2]:
DB_FAISS_PATH = 'vectorstores/db_faiss'

custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

def custom_prompt():
    prompt = PromptTemplate(template=custom_prompt_template, 
                            input_variables=['context','question'])
    return prompt

def retrieval_qa(llm,prompt,db):
    retrieval = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type="stuff",
        retriever = db.as_retriever(search_kwargs={'k':2}),
        return_source_documents=True,
        chain_type_kwargs = {"prompt":prompt}
    )
    return retrieval

def load_llm():
    llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML",
                        model_type="llama",
                        max_new_tokens=512,
                        temperature=0.5)
    return llm
    
def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={"device":"cpu"})
    db = FAISS.load_local(DB_FAISS_PATH,embeddings,allow_dangerous_deserialization=True)
    llm = load_llm()
    qa_prompt = custom_prompt()
    qa = retrieval_qa(llm, qa_prompt, db)
    return qa

def final_result():
    qa_result = qa_bot()
    response = qa_result({'query': "Explain The Sentiment, corn net income drop 13 share fall and State why It is Negative Sentiment"})
    return response

final_result()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2024-09-03 22:33:04 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


/home/vulcan/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


2024-09-03 22:33:09 - Loading faiss with AVX2 support.
2024-09-03 22:33:09 - Successfully loaded faiss with AVX2 support.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

/home/vulcan/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': 'Explain The Sentiment, corn net income drop 13 share fall and State why It is Negative Sentiment',
 'result': "Sentiment refers to the emotional response of investors towards certain asset characteristics. It can have a negative impact on asset values if investors become risk-averse or decide that there's little point in holding assets due to poor performance. In this case, the 13 share fall in corn net income is likely a result of negative sentiment among investors regarding the performance of the company.",
 'source_documents': [Document(page_content='sentiment shifts.4 This leads researchers from the behaviouralist side of the fence to make the \nclaim that the impact of sentiment on asset values is an irrefutable fact of life. The only issue \nremaining is how to measure it (Baker and Wurgler 2007). Such strong statements seem to be \nexaggerated, as sentiment is only one of a list of possible explanations for various puzzles \nobserved in the literature. Moreover, just 